In [1]:
import sys
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision import models
from PIL import Image
import matplotlib.pyplot as plt
from tabulate import tabulate
from tqdm import tqdm

# Assuming the path to your RISE-SFL folder is set up correctly
rise_sfl_path = r"C:\Users\eagua\OneDrive\Documents\GitHub\6CCS3COV-7CCSMCVI-Computer-Vision\RISEX"
sys.path.append(rise_sfl_path)
# Assuming these imports work as in your original script
from explanations import RISE
from utils import read_tensor, get_class_name, tensor_imshow

rise_sfl_path = r"C:\Users\eagua\OneDrive\Documents\GitHub\6CCS3COV-7CCSMCVI-Computer-Vision\metrics-saliency-maps"  # Replace with the actual path to your RISE-SFL folder
sys.path.append(rise_sfl_path)
#saliency_maps = extract_specific_saliency_maps(pixel_datasets[0])
from saliency_maps_metrics.multi_step_metrics import Deletion, Insertion, compute_correlation, compute_auc_metric
from saliency_maps_metrics.data_replace import select_data_replace_method


# Make sure this import works with your file structure

print("Script started")

# Set up CUDA
import torch.backends.cudnn as cudnn
cudnn.benchmark = True

# Load the model
model = models.resnet50(True)
model = nn.Sequential(model, nn.Softmax(dim=1))
model = model.eval()
model = model.cuda()

for p in model.parameters():
    p.requires_grad = False

print("Model loaded")

# Initialize RISE explainer
explainer = RISE(model, (224, 224))
explainer.load_masks(100, 0.2,'mask.npy')  # This should load masks with shape (19, 10, 1, 224, 224)

print("RISE explainer initialized")

# Set up metrics
deletion_metric = Deletion(data_replace_method="black", bound_max_step=True, batch_size=1, max_step_nb=14*14, cumulative=True)
insertion_metric = Insertion(data_replace_method="blur", bound_max_step=True, batch_size=1, max_step_nb=14*14, cumulative=True)

# Process all images in the folder
image_folder = r'C:\Users\eagua\OneDrive\Documents\GitHub\6CCS3COV-7CCSMCVI-Computer-Vision\RISE-SFL\data_demo'

print(f"Checking image folder: {image_folder}")
print(f"Folder exists: {os.path.exists(image_folder)}")
print(f"Folder contents: {os.listdir(image_folder)}")

image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

print(f"Found {len(image_files)} images in the folder")
print(f"Image files: {image_files}")

# Function to calculate sparsity
def calculate_sparsity(saliency_map):
    max_value = np.max(saliency_map)
    mean_value = np.mean(saliency_map)
    return max_value / mean_value if mean_value != 0 else float('inf')

# Initialize dictionary to store cumulative results
cumulative_results = {
    "RISE": {"DAUC": 0, "DC": 0, "IAUC": 0, "IC": 0, "Sparsity": 0}
}

# Process each image
for image_index, image_file in enumerate(tqdm(image_files, desc="Processing images")):
    img_path = os.path.join(image_folder, image_file)
    img = read_tensor(img_path).cuda()

    # Get prediction
    with torch.no_grad():
        output = model(img)
    target_class = output.argmax().item()

    # Generate saliency map
    sal_og = explainer(img, image_index=image_index)[target_class].cpu().numpy()

    # Calculate metrics
    saliency_map_tensor = torch.from_numpy(sal_og).float().unsqueeze(0).unsqueeze(0).cuda()

    # Compute Deletion metrics
    deletion_result = deletion_metric(model, img, saliency_map_tensor, class_to_explain_list=[target_class])
    cumulative_results["RISE"]["DAUC"] += deletion_result["dauc"]
    cumulative_results["RISE"]["DC"] += deletion_result["dc"]

    # Compute Insertion metrics
    insertion_result = insertion_metric(model, img, saliency_map_tensor, class_to_explain_list=[target_class])
    cumulative_results["RISE"]["IAUC"] += insertion_result["iauc"]
    cumulative_results["RISE"]["IC"] += insertion_result["ic"]

    # Calculate sparsity
    cumulative_results["RISE"]["Sparsity"] += calculate_sparsity(sal_og)

    # Visualize and save saliency map
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    tensor_imshow(img[0].cpu())
    plt.title(f"Original - {get_class_name(target_class)}")
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    tensor_imshow(img[0].cpu())
    plt.imshow(sal_og, cmap='jet', alpha=0.5)
    plt.title(f"Saliency Map (Image {image_index + 1})")
    plt.axis('off')
    
    plt.savefig(f"saliency_{image_file}_image{image_index + 1}.png")
    plt.close()

# Calculate averages
num_images = len(image_files)
results = []
for name, metrics in cumulative_results.items():
    avg_metrics = [name] + [metrics[key] / num_images for key in ["DAUC", "DC", "IAUC", "IC", "Sparsity"]]
    results.append(avg_metrics)

# Create a table using tabulate
headers = ["Saliency Map", "Avg DAUC", "Avg DC", "Avg IAUC", "Avg IC", "Avg Sparsity"]
table = tabulate(results, headers=headers, tablefmt="grid", floatfmt=".4f")

# Print the table
print(f"\nAverage Metrics for {num_images} images:")
print(table)

print("Script completed")

Script started


c:\Users\eagua\anaconda3\envs\torch_cuda\Lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
c:\Users\eagua\anaconda3\envs\torch_cuda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model loaded
RISE explainer initialized
Checking image folder: C:\Users\eagua\OneDrive\Documents\GitHub\6CCS3COV-7CCSMCVI-Computer-Vision\RISE-SFL\data_demo
Folder exists: True
Folder contents: ['.DS_Store', 'catdog.png', 'golden-retriever-puppy.jpg', 'goldfish.jpg', 'ILSVRC2012_val_00001143.JPEG', 'ILSVRC2012_val_00001209.JPEG', 'ILSVRC2012_val_00003169.JPEG', 'ILSVRC2012_val_00005140.JPEG', 'ILSVRC2012_val_00006505.JPEG', 'ILSVRC2012_val_00007455.JPEG', 'ILSVRC2012_val_00010495.JPEG', 'ILSVRC2012_val_00012187.JPEG', 'ILSVRC2012_val_00021056.JPEG', 'ILSVRC2012_val_00021591.JPEG', 'ILSVRC2012_val_00021821.JPEG', 'ILSVRC2012_val_00022112.JPEG', 'ILSVRC2012_val_00024126.JPEG', 'ILSVRC2012_val_00026600.JPEG', 'ILSVRC2012_val_00027160 copy.JPEG', 'ILSVRC2012_val_00030127.JPEG', 'ILSVRC2012_val_00035705.JPEG', 'ILSVRC2012_val_00038873.JPEG', 'ILSVRC2012_val_00039797.JPEG', 'ILSVRC2012_val_00047413.JPEG', 'ILSVRC2012_val_00048952.JPEG']
Found 24 images in the folder
Image files: ['catdog.png

Processing images: 100%|██████████| 24/24 [00:49<00:00,  2.08s/it]


Average Metrics for 24 images:
+----------------+------------+----------+------------+----------+----------------+
| Saliency Map   |   Avg DAUC |   Avg DC |   Avg IAUC |   Avg IC |   Avg Sparsity |
+================+============+==========+============+==========+================+
| RISE           |     0.2103 |   0.1824 |     0.4633 |  -0.0048 |         1.4215 |
+----------------+------------+----------+------------+----------+----------------+
Script completed


RISE ORIGINAL

In [3]:
import sys
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision import models
from PIL import Image
import matplotlib.pyplot as plt
from tabulate import tabulate
from tqdm import tqdm



# Set up the path to your RISE-SFL folder
rise_sfl_path = r"C:\Users\eagua\OneDrive\Documents\GitHub\6CCS3COV-7CCSMCVI-Computer-Vision\Causal-AI"
sys.path.append(rise_sfl_path)

# Assuming these imports work as in your original script
from explanations import RISE
from utils import read_tensor, get_class_name, tensor_imshow
print("Script started")
import torch.backends.cudnn as cudnn
cudnn.benchmark = True
model = models.resnet50(True)
model = nn.Sequential(model, nn.Softmax(dim=1))
model = model.eval()
model = model.cuda()

for p in model.parameters():
    p.requires_grad = False


print("Model loaded")

# Initialize RISE explainer
explainer = RISE(model, (224, 224))
N, s, p1 = 100, 8, 0.2
explainer.generate_masks(N, s, p1)
explainer.load_masks('masks.npy',p1,s)

print("RISE explainer initialized")

# Process all images in the folder
image_folder = r'C:\Users\eagua\OneDrive\Documents\GitHub\6CCS3COV-7CCSMCVI-Computer-Vision\RISE-SFL\data_demo'

print(f"Checking image folder: {image_folder}")
print(f"Folder exists: {os.path.exists(image_folder)}")
print(f"Folder contents: {os.listdir(image_folder)}")

image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

print(f"Found {len(image_files)} images in the folder")
print(f"Image files: {image_files}")

# Function to calculate sparsity
def calculate_sparsity(saliency_map): 
    max_value = np.max(saliency_map)
    mean_value = np.mean(saliency_map)
    return max_value / mean_value if mean_value != 0 else float('inf')

rise_sfl_path = r"C:\Users\eagua\OneDrive\Documents\GitHub\6CCS3COV-7CCSMCVI-Computer-Vision\metrics-saliency-maps"  # Replace with the actual path to your RISE-SFL folder
sys.path.append(rise_sfl_path)
#saliency_maps = extract_specific_saliency_maps(pixel_datasets[0])

from saliency_maps_metrics.multi_step_metrics import Deletion, Insertion, compute_correlation, compute_auc_metric
from saliency_maps_metrics.data_replace import select_data_replace_method

deletion_metric = Deletion(data_replace_method="black", bound_max_step=True, batch_size=1, max_step_nb=14*14, cumulative=True)
insertion_metric = Insertion(data_replace_method="blur", bound_max_step=True, batch_size=1, max_step_nb=14*14, cumulative=True)


# Initialize dictionary to store cumulative results
cumulative_results = {
    "RISE": {"DAUC": 0, "DC": 0, "IAUC": 0, "IC": 0, "Sparsity": 0}
}

# Process each image
for image_file in tqdm(image_files, desc="Processing images"):
    img_path = os.path.join(image_folder, image_file)
    img = read_tensor(img_path).cuda()

    # Get prediction
    with torch.no_grad():
        output = model(img)
    target_class = output.argmax().item()

    # Generate saliency map
    sal_og = explainer(img)[target_class].cpu().numpy()

    # Calculate metrics
    saliency_map_tensor = torch.from_numpy(sal_og).float().unsqueeze(0).unsqueeze(0).cuda()

    # Compute Deletion metrics
    deletion_result = deletion_metric(model, img, saliency_map_tensor, class_to_explain_list=[target_class])
    cumulative_results["RISE"]["DAUC"] += deletion_result["dauc"]
    cumulative_results["RISE"]["DC"] += deletion_result["dc"]

    # Compute Insertion metrics
    insertion_result = insertion_metric(model, img, saliency_map_tensor, class_to_explain_list=[target_class])
    cumulative_results["RISE"]["IAUC"] += insertion_result["iauc"]
    cumulative_results["RISE"]["IC"] += insertion_result["ic"]

    # Calculate sparsity
    cumulative_results["RISE"]["Sparsity"] += calculate_sparsity(sal_og)

    # Visualize and save saliency map
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    tensor_imshow(img[0].cpu())
    plt.title(f"Original - {get_class_name(target_class)}")
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    tensor_imshow(img[0].cpu())
    plt.imshow(sal_og, cmap='jet', alpha=0.5)
    plt.title("Saliency Map")
    plt.axis('off')
    
    plt.savefig(f"saliency_{image_file}.png")
    plt.close()

# Calculate averages
num_images = len(image_files)
results = []
for name, metrics in cumulative_results.items():
    avg_metrics = [name] + [metrics[key] / num_images for key in ["DAUC", "DC", "IAUC", "IC", "Sparsity"]]
    results.append(avg_metrics)

# Create a table using tabulate
headers = ["Saliency Map", "Avg DAUC", "Avg DC", "Avg IAUC", "Avg IC", "Avg Sparsity"]
table = tabulate(results, headers=headers, tablefmt="grid", floatfmt=".4f")

# Print the table
print(f"\nAverage Metrics for {num_images} images:")
print(table)

print("Script completed")

Script started


c:\Users\eagua\anaconda3\envs\torch_cuda\Lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
c:\Users\eagua\anaconda3\envs\torch_cuda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model loaded


Generating filters: 100%|██████████| 100/100 [00:00<00:00, 889.73it/s]


RISE explainer initialized
Checking image folder: C:\Users\eagua\OneDrive\Documents\GitHub\6CCS3COV-7CCSMCVI-Computer-Vision\RISE-SFL\data_demo
Folder exists: True
Folder contents: ['.DS_Store', 'catdog.png', 'golden-retriever-puppy.jpg', 'goldfish.jpg', 'ILSVRC2012_val_00001143.JPEG', 'ILSVRC2012_val_00001209.JPEG', 'ILSVRC2012_val_00003169.JPEG', 'ILSVRC2012_val_00005140.JPEG', 'ILSVRC2012_val_00006505.JPEG', 'ILSVRC2012_val_00007455.JPEG', 'ILSVRC2012_val_00010495.JPEG', 'ILSVRC2012_val_00012187.JPEG', 'ILSVRC2012_val_00021056.JPEG', 'ILSVRC2012_val_00021591.JPEG', 'ILSVRC2012_val_00021821.JPEG', 'ILSVRC2012_val_00022112.JPEG', 'ILSVRC2012_val_00024126.JPEG', 'ILSVRC2012_val_00026600.JPEG', 'ILSVRC2012_val_00027160 copy.JPEG', 'ILSVRC2012_val_00030127.JPEG', 'ILSVRC2012_val_00035705.JPEG', 'ILSVRC2012_val_00038873.JPEG', 'ILSVRC2012_val_00039797.JPEG', 'ILSVRC2012_val_00047413.JPEG', 'ILSVRC2012_val_00048952.JPEG']
Found 24 images in the folder
Image files: ['catdog.png', 'golden-re

Processing images: 100%|██████████| 24/24 [00:47<00:00,  1.96s/it]


Average Metrics for 24 images:
+----------------+------------+----------+------------+----------+----------------+
| Saliency Map   |   Avg DAUC |   Avg DC |   Avg IAUC |   Avg IC |   Avg Sparsity |
+================+============+==========+============+==========+================+
| RISE           |     0.1032 |   0.2321 |     0.5791 |   0.0523 |         2.4014 |
+----------------+------------+----------+------------+----------+----------------+
Script completed
